[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/vizier/blob/main/docs/guides/developer/using_designers.ipynb)

### GP Bandit Designer Example

The following example, which uses Vizier's GP Bandit designer, shows how to interact with Vizier designers.

In [ ]:
from vizier import pyvizier as vz
from vizier._src.algorithms.designers import gp_bandit

# The problem statement (which parameters are being optimized)
problem = vz.ProblemStatement()
problem.search_space.root.add_float_param('x', 0.0, 1.0)
problem.search_space.root.add_float_param('y', 0.0, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='maximize_metric', goal=vz.ObjectiveMetricGoal.MAXIMIZE))

# Create a new designer object
designer = gp_bandit.VizierGPBandit(problem)
# Ask the designer for 5 suggestions
suggestions = designer.suggest(count=2)

In this case, since the designer was not update with any `COMPLETED` or `ACTIVE` trials, it will produce suggestions which will look like:

```python
[TrialSuggestion(parameters=ParameterDict(_items={'x': 0.5, 'y': 0.5}), metadata=Metadata((namespace:, items: {'seeded': 'center'}), current_namespace=)),
 TrialSuggestion(parameters=ParameterDict(_items={'x': 0.10274669379450661, 'y': 0.10191725529767912}), metadata=Metadata((namespace:, items: {}), current_namespace=))]
 ```

Note that the first suggestion is seeded at the center of the search space, and the second suggestion is random.  If we call `designer.suggest()` again, before calling `update()`, the designer will produce an identical first suggestion at the center of the search space, and a second random suggestion.

Only when we call `update()`, will the designer update its internal state and generate different suggestions:

In [ ]:
import numpy as np
from vizier import algorithms as vza
from vizier import pyvizier as vz
from vizier._src.algorithms.designers import gp_bandit

# The problem statement (which parameters are being optimized)
problem = vz.ProblemStatement()
problem.search_space.root.add_float_param('x', 0.0, 1.0)
problem.search_space.root.add_float_param('y', 0.0, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='maximize_metric', goal=vz.ObjectiveMetricGoal.MAXIMIZE))

# Create a new designer object
designer = gp_bandit.VizierGPBandit(problem)
# Ask the designer for 5 suggestions
suggestions = designer.suggest(count=2)

completed_trials = []
for suggestion in suggestions:
  metric_value = np.random.random()  # Make up a fake metric value.
  suggestion.complete(vz.Measurement(metrics={'maximize_metric': metric_value})))

# Update the designer with the completed trials.
designer.update(vza.CompletedTrials(completed_trials), vza.ActiveTrials())

# Ask for more suggestions.
new_suggestions = designer.suggest(count=2)

Note that `COMPLETED` trials should be incrementally updated, while all `ACTIVE` trials are passed to the designer in every `update()` call.

`Designer` can also be seeded with pre-existing data.

Consider the following example:

In [ ]:
from vizier import algorithms as vza
from vizier import pyvizier as vz
from vizier._src.algorithms.designers import gp_bandit

# The problem statement (which parameters are being optimized)
problem = vz.ProblemStatement()
problem.search_space.root.add_float_param('x', 0.0, 1.0)
problem.search_space.root.add_float_param('y', 0.0, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='maximize_metric', goal=vz.ObjectiveMetricGoal.MAXIMIZE))

designer = gp_bandit.VizierGPBandit(problem)

# Create completed trials representing pre-existing training data.
trials = [vz.Trial(parameters={'x': 0.5, 'y': 0.6}).complete(vz.Measurement(metrics={'maximize_metric': 0.3}))]
designer.update(vza.CompletedTrials(trials), vza.ActiveTrials())

# As the designer for suggestions.
suggestions = designer.suggest(count=2)

In this case, the designer will not return a first trial seeded at the center of the search space, since it has been updated with completed trials. The new suggestions will look something like:

```python
[TrialSuggestion(parameters=ParameterDict(_items={'x': 0.7199945005054509, 'y': 0.3800034493548722}), ...]
```